Pseudocódigo GCL



*   Mf significa el numero filtrado de datos de la base de datos respecto a unafiltración
*   writeln es la equivalencia a un sout en Java o print en python pero en pseudocódigo


```pseudocode

var data : array[0,M) x [0,N) of array [0,k] of char ;
var columns : array[0,N) of array[0,k] of char ;
var df : array[0,M) x [0,N) of array [0,k] of char ;

var tipo, combustible : array[0,k] of char ;
var consumo, precio_min, precio_max, maletero : int ;

var df_filtrado : array[0,Mf) x [0,N) of array [0,k] of char;
var df_scaled : array[0,Mf) x [0,N) of float ;

var usuario_vector  : array[0,3) of float ;
var autos_vector    : array[0,Mf) x [0,3) of float ;   
var distancias      : array[0,Mf) of float ;
var similitudes     : array[0,Mf) of float ;

var i, j, k, m : int ;
var boton : bool ;
var recomendaciones : array[0,R) of array[0,k] of char;

var peso_consumo, peso_precio, peso_maletero : float;
var pesos : array[0,3) of float;

boton := true;

data := [
    ["Toyota","Corolla","Sedán","Gasolina",16,23000,470,"FWD","Intermedia"],
    ["Ford","Ranger","Pickup","Diesel",10,38000,900,"4x4","Intermedia"],
    ["Tesla","Model 3","Sedán","Eléctrico",18,42000,425,"RWD","Alta"],
    ["Chevrolet","Spark","Hatchback","Gasolina",19,13500,170,"FWD","Básica"],
    ["Hyundai","Tucson","SUV","Gasolina",13,29000,620,"AWD","Intermedia"],
    ["Kia","Rio","Sedán","Gasolina",17,16000,390,"FWD","Básica"],
    ["BMW","X5","SUV","Diesel",9,65000,650,"AWD","Alta"],
    ["Audi","A4","Sedán","Gasolina",12,46000,480,"AWD","Alta"],
    ["Nissan","Leaf","Hatchback","Eléctrico",16,33000,380,"FWD","Intermedia"],
    ["Jeep","Wrangler","SUV","Gasolina",8,44000,900,"4x4","Alta"],
    ["Honda","Civic","Sedán","Gasolina",15,24000,450,"FWD","Intermedia"],
    ["Mazda","CX-5","SUV","Gasolina",12,31000,550,"AWD","Intermedia"],
    ["Renault","Duster","SUV","Diesel",14,22000,475,"FWD","Básica"],
    ["Mercedes-Benz","GLC","SUV","Gasolina",11,57000,600,"AWD","Alta"],
    ["Volkswagen","Golf","Hatchback","Híbrido",18,25000,380,"FWD","Intermedia"]
];
columns := ["Marca","Modelo","Tipo","Combustible","Consumo","Precio","Maletero","Tracción","Tecnología"];
df := data;


#Inputs de usuario
writeln("🔍 Recomendador de Autos con KNN");

writeln("Seleccione tipo de auto (Sedán / SUV / Pickup / Hatchback): ");
readln(tipo);

writeln("Seleccione tipo de combustible (Gasolina / Diesel / Eléctrico / Híbrido): ");
readln(combustible);

writeln("Ingrese consumo mínimo (km/L): ");
readln(consumo);

writeln("Ingrese rango de precio mínimo: ");
readln(precio_min);

writeln("Ingrese rango de precio máximo: ");
readln(precio_max);

writeln("Ingrese capacidad mínima del maletero (L): ");
readln(maletero);

writeln("Defina peso de Consumo (0.1 - 5.0): ");
readln(peso_consumo);

writeln("Defina peso de Precio (0.1 - 5.0): ");
readln(peso_precio);

writeln("Defina peso de Maletero (0.1 - 5.0): ");
readln(peso_maletero);

pesos[0] := peso_consumo;
pesos[1] := peso_precio;
pesos[2] := peso_maletero;

writeln("Ingrese número de recomendaciones k: ");
readln(k);

#Filtrar dataset
i := 0; j := 0;
do i < M ->
    if df[i,"Tipo"] = tipo ∧ df[i,"Combustible"] = combustible ∧
       df[i,"Precio"] >= precio_min ∧ df[i,"Precio"] <= precio_max ->
        df_filtrado[j] := df[i];
        j := j + 1
    [] -> skip
    fi;
    i := i + 1
od;

#Sistema de recomendación con KNN
if boton = true ->
    if j = 0 ->
        writeln("❌ No se encontraron autos que cumplan los filtros obligatorios.")
    [] ->
        # Normalizar datos
        normalizar(df_filtrado[,"Consumo"], df_scaled[,"Consumo"]);
        normalizar(df_filtrado[,"Precio"], df_scaled[,"Precio"]);
        normalizar(df_filtrado[,"Maletero"], df_scaled[,"Maletero"]);

        # Vector del usuario
        usuario_vector[0] := normalizar_valor(consumo, df_filtrado[,"Consumo"]);
        usuario_vector[1] := normalizar_valor((precio_min+precio_max)/2, df_filtrado[,"Precio"]);
        usuario_vector[2] := normalizar_valor(maletero, df_filtrado[,"Maletero"]);

        # Aplicar pesos
        i := 0;
        do i<3 ->
            usuario_vector[i] := usuario_vector[i] * pesos[i];
            i := i + 1
        od;

        i := 0;
        do i<j ->
            autos_vector[i,0] := df_scaled[i,"Consumo"] * pesos[0];
            autos_vector[i,1] := df_scaled[i,"Precio"] * pesos[1];
            autos_vector[i,2] := df_scaled[i,"Maletero"] * pesos[2];
            i := i + 1
        od;

        # Ajustar k si hay menos autos que el solicitado
        if j < k ->
            k := j
        [] -> skip
        fi;

        # Calcular distancias euclidianas
        i := 0;
        do i<j ->
            distancias[i] := euclidean_distance(usuario_vector, autos_vector[i]);
            similitudes[i] := 1 / (1 + distancias[i]);
            i := i + 1
        od;

        # Ordenar por similitud descendente
        ordenar_por(similitudes, df_filtrado);

        # Mostrar resultados
        i := 0;
        do i<k ->
            writeln("🚘 ", df_filtrado[i,"Marca"], " ", df_filtrado[i,"Modelo"],
                    " Similitud: ", similitudes[i]);
            i := i + 1
        od;
    fi
[] boton = false ->
    skip
fi;



```


```
